In [72]:
import pandas as pd
from assign_unique_ids_functions import *
pd.options.display.max_columns = 999
import numpy as np
uhc = ['UID', 'Unit', 'Date', 'Event']

In [73]:
um1 = pd.read_csv('../input/ase-units.csv.gz')
um1.dropna(thresh=3, axis=0, inplace=True)
um1.rename(columns={'Effective.Date' : 'Start.Date'}, inplace=True)
um1.drop('ase-units_ID', inplace=True, axis=1)
um1.loc[um1['End.Date'] < um1['Start.Date'], 'End.Date'] = np.nan
um1.drop_duplicates(inplace=True)
um1.head()

,Unit,Start.Date,End.Date,UID
0,14.0,2007-03-29,NaN,1
2,15.0,2007-03-29,NaN,2
4,1.0,1976-08-19,1976-08-20,3
5,4.0,1971-01-16,1976-08-18,3
6,22.0,1970-06-15,1971-01-15,3


In [74]:
um2 = pd.read_csv('../input/all-sworn-units.csv.gz')
um2.dropna(thresh=3, axis=0, inplace=True)
um2.drop('all-sworn-units_ID', inplace=True, axis=1)
um2.loc[um2['End.Date'] < um2['Start.Date'], 'End.Date'] = np.nan
um2.drop_duplicates(inplace=True)
um2.head()

,Unit,Start.Date,End.Date,UID
0,3.0,2016-06-05,NaN,1
1,14.0,2007-03-29,NaN,1
2,14.0,2007-03-29,2016-06-04,1
3,15.0,2007-03-29,NaN,2
5,1.0,1976-08-19,1976-08-20,3


In [75]:
uh = pd.concat([um1, um2]).drop_duplicates()

In [76]:
uh.head()

,Unit,Start.Date,End.Date,UID
0,14.0,2007-03-29,NaN,1
2,15.0,2007-03-29,NaN,2
4,1.0,1976-08-19,1976-08-20,3
5,4.0,1971-01-16,1976-08-18,3
6,22.0,1970-06-15,1971-01-15,3


In [77]:
prof = pd.read_csv('officer-profiles.csv.gz')[['UID', 'Resignation.Date']]

In [78]:
prof = prof[(prof['UID'].isin(um1['UID'])) & (prof['Resignation.Date'].notnull())]

In [79]:
prof.columns = ['UID', 'End.Date']

In [80]:
uh_ed = uh[uh['End.Date'].notnull()]

In [81]:
uh_ned = uh[uh['End.Date'].isnull()].drop('End.Date', axis=1)
uh_ned = uh_ned.merge(prof, on='UID', how='left')

In [82]:
uh = uh_ed.append(uh_ned).reset_index(drop=True)

In [83]:
uh.sort_values(['UID', 'Start.Date']).head()

,End.Date,Start.Date,UID,Unit
76177,2016-06-04,2007-03-29,1,14.0
77154,NaN,2007-03-29,1,14.0
110935,NaN,2016-06-05,1,3.0
77155,NaN,2007-03-29,2,15.0
2,1971-01-15,1970-06-15,3,22.0


In [84]:
uh_kd = aggregate_data(keep_duplicates(uh, ['UID','Start.Date', 'Unit']) , uid='UID', id_cols=['Start.Date', 'Unit'], max_cols = ['End.Date'])

In [85]:
uh = remove_duplicates(uh,['UID','Start.Date', 'Unit']).append(uh_kd)

In [86]:
uh.sort_values(['UID', 'Start.Date']).head()

,End.Date,Start.Date,UID,Unit
0,2016-06-04,2007-03-29,1,14.0
110935,NaN,2016-06-05,1,3.0
77155,NaN,2007-03-29,2,15.0
2,1971-01-15,1970-06-15,3,22.0
1,1976-08-18,1971-01-16,3,4.0


In [87]:
max(uh['Start.Date'].dropna())

'2016-10-09'

In [88]:
max(uh['End.Date'].dropna())

'2017-08-24'

In [89]:
uh.sort_values('End.Date', ascending=True, na_position='first').tail(5)

,End.Date,Start.Date,UID,Unit
91506,2017-08-15,2003-07-24,13548,12.0
91408,2017-08-19,2015-01-15,13454,5.0
1335,2017-08-19,2000-06-22,13454,7.0
89197,2017-08-19,1999-09-16,11359,2.0
97160,2017-08-24,2000-06-26,18928,2.0


In [90]:
pd.to_datetime(pd.date_range(start = max(uh['Start.Date'].dropna()), periods=2, closed='right'))[0]

Timestamp('2016-10-10 00:00:00', freq='D')

In [91]:
date_cap = pd.to_datetime(pd.date_range(start = max(uh['Start.Date'].dropna()), periods=2, closed='right'))[0].date()

In [92]:
uh.loc[(uh['End.Date'].isnull()) | 
       (pd.to_datetime(uh['End.Date']) > date_cap),
  'End.Date'] = pd.to_datetime(date_cap)
uh['End.Date']= pd.to_datetime(uh['End.Date'])

In [100]:
uh.reset_index(drop=True, inplace=True)

In [101]:
uh.head()

,End.Date,Start.Date,UID,Unit
0,2016-10-10,2016-06-05,1,3.0
1,2016-10-10,2007-03-29,2,15.0
2,1971-01-15,1970-06-15,3,22.0
3,1976-08-18,1971-01-16,3,4.0
4,1976-08-20,1976-08-19,3,1.0


In [94]:
uh2 = pd.DataFrame()

In [115]:
for i,row in uh.iterrows():
    tdf = pd.DataFrame(pd.date_range(start=row['Start.Date'], end=row['End.Date'], name='Date'))
    tdf['Unit'] = row['Unit']
    tdf['UID'] = row['UID']
    uh2 = uh2.append(tdf)

KeyboardInterrupt: 